# Tutorial for data analysis using PySpark

This tutorial is based on the LinkedIn Learning example "Apache PySpark by Example" by Jonathan Fernandes.

The first steps needed to run this tutorial comprise installing PySpark, creating a Spark Session and downloading the data (to the virtual environment).

In [ ]:
!pip install pyspark==3.5.1

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
!ls -l

In [ ]:
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv
!ls -l

In [ ]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5